In [1]:
import numpy as np
import scipy.io as sio
from scipy.ndimage import gaussian_filter

# PDE solver
def solve_pde(u, sigma, iterations=4000):
    H, W = sigma.shape
    u_old = np.zeros_like(u)

    for it in range(iterations):
        u_old[:] = u

        for i in range(1, H-1):
            for j in range(1, W-1):

                if j == 0 or j == W-1:
                    continue

                sxp = 0.5 * (sigma[i,j] + sigma[i,j+1])
                sxm = 0.5 * (sigma[i,j] + sigma[i,j-1])
                syp = 0.5 * (sigma[i,j] + sigma[i+1,j])
                sym = 0.5 * (sigma[i,j] + sigma[i-1,j])

                u[i,j] = (
                    sxp * u[i,j+1] +
                    sxm * u[i,j-1] +
                    syp * u[i+1,j] +
                    sym * u[i-1,j]
                ) / (sxp + sxm + syp + sym + 1e-12)

        # Neumann BC at top/bottom
        u[0,:]  = u[1,:]
        u[-1,:] = u[-2,:]

        # convergence
        if np.max(np.abs(u - u_old)) < 1e-6:
            break

    return u


# Ramdom smooth conductivity
def generate_random_sigma(H, W, mask):
    sigma = np.random.rand(H, W)
    sigma = gaussian_filter(sigma, sigma=5)
    sigma = 0.5 + 1.5 * sigma   
    sigma *= mask         
    return sigma

 
def generate_dataset(N=10, maskfile="conductor_data.mat"):

    mask = sio.loadmat(maskfile)["mask_crop"]
    H, W = mask.shape
    print("Loaded mask shape:", mask.shape)

    for k in range(N):
        sigma = generate_random_sigma(H, W, mask)

        u0 = np.zeros((H, W))
        u0[:, 0]  = 1.0
        u0[:, -1] = 0.0

        U_sim = solve_pde(u0, sigma)

        sio.savemat(f"sigma_{k}.mat", {"sigma": sigma})
        sio.savemat(f"U_sim_{k}.mat", {"U_sim": U_sim})

        print(f"Saved sigma_{k}.mat and U_sim_{k}.mat")

    print("\nFinished generating synthetic dataset!")


# Run generator
generate_dataset(N=10)


Loaded mask shape: (66, 65)
Saved sigma_0.mat and U_sim_0.mat
Saved sigma_1.mat and U_sim_1.mat
Saved sigma_2.mat and U_sim_2.mat
Saved sigma_3.mat and U_sim_3.mat
Saved sigma_4.mat and U_sim_4.mat
Saved sigma_5.mat and U_sim_5.mat
Saved sigma_6.mat and U_sim_6.mat
Saved sigma_7.mat and U_sim_7.mat
Saved sigma_8.mat and U_sim_8.mat
Saved sigma_9.mat and U_sim_9.mat

Finished generating synthetic dataset!


In [4]:
import sys
print(sys.executable)


/srv/conda/bin/python


In [5]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 24.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 34.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 50.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 36.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 59.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 59.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 55.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 53.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

#mask loaded
mask = loadmat("conductor_data.mat")["mask_crop"].astype(np.float32)
H, W = mask.shape
inside = (mask == 1)

# classes of the datasets
class SyntheticEITDataset(Dataset):
    def __init__(self, file_list):
        self.files = file_list

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):

        base = self.files[idx]           
        mat_u = loadmat(f"U_sim_{base}.mat")
        mat_s = loadmat(f"sigma_{base}.mat")

        U = mat_u["U_sim"].astype(np.float32)    
        sigma = mat_s["sigma"].astype(np.float32)    

        # Fill NaNs
        U = np.nan_to_num(U, nan=np.nanmean(U[inside]))
        sigma = np.nan_to_num(sigma, nan=np.nanmean(sigma[inside]))

        # Normalize U inside conductor region excluding graphite areas
        mean = U[inside].mean()
        std = U[inside].std() + 1e-8
        U_norm = (U - mean) / std

        # Convert to tensors 
        return (
            torch.tensor(U_norm).unsqueeze(0),    # input voltage
            torch.tensor(sigma).unsqueeze(0),      # target conductivity
            torch.tensor(mask).unsqueeze(0)         # mask
        )


# training set build
files = sorted(glob.glob("U_sim_*.mat"))

# keep only those ending in a number: U_sim_0.mat, U_sim_1.mat, ...
ids = []
for f in files:
    base = f.split("_")[-1].split(".")[0]
    if base.isdigit():      # only numeric IDs
        ids.append(base)

print("Using synthetic IDs:", ids)

dataset = SyntheticEITDataset(ids)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

print(f"Found {len(dataset)} synthetic samples.")

# 4. actual CNN architecture
class CNN_Inverse(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 1, kernel_size=5, padding=2)
        )

    def forward(self, x):
        return self.net(x)

model = CNN_Inverse().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# masked loss
def masked_mse(pred, true, mask):
    diff = (pred - true) * mask
    return torch.mean(diff**2)


# training loop 
epochs = 2000
print("Training CNN on MULTIPLE samples...")

for epoch in range(epochs):

    epoch_loss = 0

    for U_batch, sigma_batch, mask_batch in loader:
        U_batch = U_batch.to(device)
        sigma_batch = sigma_batch.to(device)
        mask_batch = mask_batch.to(device)

        optimizer.zero_grad()
        pred = model(U_batch)

        loss = masked_mse(pred, sigma_batch, mask_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d} | Loss = {epoch_loss:.4e}")


# Saving Trained model 

torch.save(model.state_dict(), "cnn_multisample.pt")
print("Saved cnn_multisample.pt")


Running on: cpu
Using synthetic IDs: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Found 10 synthetic samples.
Training CNN on MULTIPLE samples...
Epoch    0 | Loss = 2.9589e+00
Epoch  100 | Loss = 3.2275e-03
Epoch  200 | Loss = 1.9895e-03


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running trained CNN on REAL voltage. Device =", device)

# CNN 
class CNN_Inverse(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 1, kernel_size=5, padding=2)
        )
    def forward(self, x):
        return self.net(x)

# load saved CNN weights from synthetic data 
model = CNN_Inverse().to(device)
model.load_state_dict(torch.load("cnn_multisample.pt", map_location=device))
model.eval()
print("Loaded cnn_multisample.pt successfully!")

# Real experimental voltage with mask 
data = loadmat("conductor_data.mat")
U_real = data["U_conductor"] # shape is (66,65)
mask   = data["mask_crop"]

U_real = np.nan_to_num(U_real, nan=np.nanmean(U_real))
print("Loaded U_real shape:", U_real.shape)

# normalization of real voltage 
U_mean = U_real[mask == 1].mean()
U_std  = U_real[mask == 1].std() + 1e-8

U_real_norm = (U_real - U_mean) / U_std

# running prediction
U_input = torch.tensor(U_real_norm, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

with torch.no_grad():
    sigma_pred_real = model(U_input).cpu().numpy()[0, 0]

#FInal save 
savemat("sigma_pred_real.mat", {"sigma_pred_real": sigma_pred_real})
print("Saved sigma_pred_real.mat")

# visualize

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.title("Real Voltage U_real (normalized)")
plt.imshow(U_real_norm, cmap="bwr")
plt.colorbar()

plt.subplot(1,2,2)
plt.title("CNN Predicted σ(x,y) on REAL voltage")
plt.imshow(sigma_pred_real, cmap = "bwr")
plt.colorbar()

plt.tight_layout()
plt.show()

